# Using unlabelled, unfractionated datasets obtained from QExact and VOrbi instruments

In [5]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [6]:
import Classification_Utils as cu
#import MaxQuant_Postprocessing_Functions as mq
import pandas as pd

## Load (and combine?) data from all tissues

In [4]:
# Loop through files, create dataframe, join to existing

#file_paths = ['D:\file1.txt', 'D:\file2.txt']

### Break out into re-usable function

"""
Args:
    file_paths (list of strings): list of file paths to csvs 
    
Returns:
    dataframe containing data from all csvs referenced by file_paths. Dataframe index is 'Peptide'; each column represents a single sample.
"""
def combine_csvs(file_paths):
    
    dfs = []

    for file in file_paths:
        df = pd.read_csv(file, sep='\t', lineterminator='\r')
        dfs.append(df)

    combined_df = pd.DataFrame()
    for df in dfs:
        df.set_index('Peptide', inplace=True)
        combined_df = combined_df.join(df)
        
    return combined_df
    

## Separate features so that future data can be fit to the correct shape and classified
* Save array/dataframe of features (via pickle?)
* Write script to classify new data-- load features and fit new data on them

## Clean data
* Log2 transform
* Mean/Median normalize
* Impute missing values

## Map each column to a corresponding label

## Visualize data
* Scree plot
* PCA plot
* Pearson Matrix

## Test various classifiers using cross-validation

### Decision Tree

### KNN

### Logistic Regression

### Naive Bayes

### SVC variations

### Aggregations
* Decision Tree
* Gradient Boosting

## Tune parameters of best models (if applicable)
* Check accuracy score and F1 score (measure of precision and recall)

##  Confusion matrices of best models

## Top expressed proteins/peptides per tissue